In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import math

In [ ]:
def openfile(fileName):
    """
    Dado el nombre de un fichero devuelve el corpus preprocesado y las etiquetas
    """
    with open(fileName,"rb") as fh:
        texto,etiquetas = pickle.load(fh)
    return texto,etiquetas

In [ ]:
def saveRep(representacion,etiquetas, outputFile):
    """
    Vuelca la representación y las etiquetas en un fichero
    """
    with open(outputFile,"wb") as fh:
        object = representacion,etiquetas
        pickle.dump(object,fh)

In [ ]:
#Creamos un Word2Vec a partir de los tweets extraidos de Streamings de Twitter y corpus del PAN 
#CUESTA MUCHO TIEMPO DE COMPUTO ¡MEJOR COMENTARLO :/ !

#data = pd.read_csv("datosParaWord2VecProcesado.csv")
#corpusWord2Vec = textoTrain
#corpusToken = [x.split() for x in corpusWord2Vec]
#corpusToken = [x.split() for x in data["Data"] ]
#model = Word2Vec(corpusToken, size=300, window=5, min_count=1, workers=2)
print("modelo entrenado")

In [ ]:
#Cargamos el modelo pre-entrenado
with open("word2vecmodel","rb") as fh:
    word2vec = pickle.load(fh)
word_vectors = word2vec.wv

In [ ]:
word2vec.most_similar("politico") #Palabras con un contexto similar a político

In [ ]:
textoTrain, etiquetasTrain = openfile("preproceso_train")
textoTest, etiquetasTest = openfile("preproceso_test")
print("Lectura de ficheros correcta. Documentos de train: %d Documentos de test: %d " %(len(textoTrain),len(textoTest)))

In [ ]:
#Train
corpusTrainw2v = []
keysNoExistentes = 0
documentosprocesados = 0
for document in textoTrain:
    #Por cada autor
    wordlist = []
    for word in document:
        #obtenemos el embedding de cada una de las palabras
        try:
            wordlist.append(word_vectors[word].tolist()) #añadimos el embedding de cada palabra
        except KeyError :
            keysNoExistentes+=1
    matrixmedia = np.matrix(wordlist)
    corpusTrainw2v.append(matrixmedia.mean(0)) #computamos la media por columnas
    documentosprocesados+=1
    if documentosprocesados % 100 == 0:
        print("Documentos procesados %d" %documentosprocesados) 
print("Keys no existentes %d" % keysNoExistentes)

In [ ]:
#Creamos una EDA valida para los algoritmos de aprendizaje TRAIN
corpusW2VTrain = []
for i in range(0,len(corpusTrainw2v)):
    corpusW2VTrain.append(corpusTrainw2v[i].tolist()[0])
print("Corpus Train OK")

In [ ]:
#Test
corpusTestw2v = []
keysNoExistentes = 0
documentosprocesados = 0
for document in textoTest:
    wordlist = []
    for word in document:
        try:
            wordlist.append(word_vectors[word].tolist())
        except KeyError :
            keysNoExistentes+=1
    matrixmedia = np.matrix(wordlist)
    corpusTestw2v.append(matrixmedia.mean(0))
    documentosprocesados+=1
    if documentosprocesados % 100 == 0:
        print("Documentos procesados %d" %documentosprocesados) 
print("Keys no existentes %d" % keysNoExistentes)

In [ ]:
#Creamos una EDA valida para los algoritmos de aprendizaje TEST
corpusW2VTest = []
for i in range(0,len(corpusTestw2v)):
    corpusW2VTest.append(corpusTestw2v[i].tolist()[0])
print("Corpus Test listo")

# Evaluación

In [ ]:
def cross_validation(clasificador,xtrain,xlabels,bloques):
    scores = cross_val_score(clasificador, xtrain, xlabels, cv=bloques, scoring='f1_macro')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    return scores.mean()

In [ ]:
def intervalo95(p,datos):
    """
    Calcula el intervalo al 95% 
    :param p: probabilidad de acierto
    :param datos: numero de datos del conjunto de test
    """
    e = 1.96*math.sqrt((p*(1-p))/datos)
    s = "[%.3f , %.3f]" % (p-e,p+e)
    return s

In [ ]:
cs = [1,10,100,1000,10000]
kernel= ["linear","rbf"]
res = "c \t \t k \t \t Accuracy \t \t Inter95 \n"
res += "---------------------SVM--------------------------\n"
for k in kernel:
    for c in cs:
        clf = svm.SVC(C = c,kernel=k)
        clf.fit(corpusW2VTrain,etiquetasTrain)
        #media = cross_validation(clf,corpusW2VTrain,etiquetasTrain,10)
        p = clf.score(corpusW2VTest,etiquetasTest)
        aux = "%d \t \t  %s \t \t %.4f \t \t %s \n" % (c,k[0:3],p, intervalo95(p,len(etiquetasTest)))
        print(aux)
        res+= aux
print(res)

In [ ]:
cs = [1,10,100,1000,10000]
res = "c \t \t Accuracy \t \t Inter95 \n"
res += "---------------------Logistic--------------------------\n"
for c in cs:
    clf = LogisticRegression(C=c)
    clf.fit(corpusW2VTrain,etiquetasTrain)
    #media = cross_validation(clf,corpusW2VTrain,etiquetasTrain,10)
    p = clf.score(corpusW2VTest,etiquetasTest)
    aux = "%d \t \t %.3f \t \t %s \n" %(c,p,intervalo95(p,len(etiquetasTest)))
    print(aux)
    res+= aux
print(res)

In [ ]:
saveRep(corpusW2VTrain,etiquetasTrain,"w2vRepTrain")
saveRep(corpusW2VTest,etiquetasTest,"w2vRepTest")